In [1]:
import os
import corenlp 

os.environ["CORENLP_HOME"] = r'C:\stanford-corenlp-full-2018-10-05'
client = corenlp.CoreNLPClient()

def get_nouns(sentence, chunk = False):
    text = sentence.lower()
    res = []
    try:
        pattern = '[{pos:NN}] | [{pos:NNS}] | [{pos:NNP}] | [{pos:NNPS}]'
        matches = client.tokensregex(sentence, pattern)
        response = matches['sentences'][0]
        len_chunk = response['length']
        if len_chunk > 0:
            for index in range(0, len_chunk):
                res.append(response[str(index)]['text'])
    except:
        print('err : ', sentence)
        res = []
    return res

def get_adverbs(sentence, chunk = False):
    text = sentence.lower()
    res = []
    try:
        pattern = '[{pos:RB}]'
        matches = client.tokensregex(sentence, pattern)
        response = matches['sentences'][0]
        len_chunk = response['length']
        if len_chunk > 0:
            for index in range(0, len_chunk):
                res.append(response[str(index)]['text'])
    except:
        print('err')
        res = []
    return res

In [2]:
import pandas as pd
from tqdm import tqdm_notebook

def run(tipe):
    df = pd.read_csv("Results/ATE/"+ tipe +".csv")
    sf = pd.DataFrame(columns = ['id','review','target', 'category','term', 'polarity'])
    count = 0
    for index in tqdm_notebook(range(len(df))):
        new_term = []
        if type(df['term'][index]) != float:
            terms = df['term'][index].split('|')
            for term in terms:
                if term not in adverbs:
                    new_term.append(term)

        if len(new_term) == 0:
            nn = get_nouns(df['review'][index])
            for n in nn:
                new_term.append(n.lower() + '!-')
            count += 1
        
        sf = sf.append({'id': df['id'][index], 
                    'review': df['review'][index],
                    'target': df['target'][index],
                    'category': df['category'][index],
                    'term': '|'.join(new_term),
                    'polarity': df['polarity'][index]
                   }, ignore_index=True)
    print(count)

    sf.to_csv("Results/add_noun/"+ tipe +"-partial.csv")
    sf.to_excel("Results/add_noun/"+ tipe +"-partial.xlsx")

In [3]:
run("dp")

err :  Not because you are "The Four Seasons"... – you are allowed to charge an arm and a leg for a romatic dinner.

60
